<center><h2>Artificial and Computational Intelligence (Assignment - 2)</h2></center>

## Problem Statement

As part of the 2nd Assignment, we'll implement Bayesian Networks and also learn to use the pomegranate library.

You are required to create a bayesian network model which would help you predict the probability. The detailed problem description is attached as a PDF as a part of this assignment along with the marking scheme.  

### What is a Bayesian Network ?

A Bayesian network, Bayes network, belief network, decision network, Bayes(ian) model or probabilistic directed acyclic graphical model is a probabilistic graphical model (a type of statistical model) that represents a set of variables and their conditional dependencies via a directed acyclic graph (DAG). 

Bayesian networks are ideal for taking an event that occurred and predicting the likelihood that any one of several possible known causes was the contributing factor. For example, a Bayesian network could represent the probabilistic relationships between diseases and symptoms. Given symptoms, the network can be used to compute the probabilities of the presence of various diseases. 

### Dataset

The dataset can be downloaded from https://drive.google.com/drive/folders/1oMtKmmvPkN4O8DmrHMJe6M8CbB93Z5kw .You can access it only using your BITS IDs. Also, the same dataset is attached along with the assignment. 

#### Dataset Description
##### Sample Tuple

Y	won	5wickets	lost	2nd	vWest_Indies	Home	6-Nov-11

##### Explanation
- The first column represents if Ashwin was in the playing 11 or not. 
- The second column represents the Result of the match . win indicates India won the match.
- The third column represents the Margin of victory / losss.
- The fourth column represents the results of the toss. won indicates India won the toss. 
- The fifth column represents the batting order. If India batted 1st or 2nd. 
- The sixth column represents the opponent.
- The seventh column represents the location of the match. If the match was held in Home(India) or away. 
- The last column represents the start date of the match.


### Evaluation
We wish to evaluate based on 
- coding practices being followed
- commenting to explain the code and logic behind doing something
- your understanding and explanation of data
- how good the model would perform

In [ ]:
# BITS RollNumbers , Names. 
2018AC04537 - Aditya Kashyap Singh
2018AC04538 - Amal Srivastava
2018AC04564 - Avinash Srivastav

In [2]:
#Import libraries
import pandas as pd
import numpy as np

In [3]:
#Read data
df = pd.read_excel (r'India_Test_Stats.xlsx')

In [4]:
df.head()

,Ashwin,Result,Margin,Toss,Bat,Opposition,Location,Start Date
0,Y,won,5 wickets,lost,2nd,v West Indies,Home,2011-11-06
1,Y,won,inns & 15 runs,won,1st,v West Indies,Home,2011-11-14
2,Y,draw,-,lost,2nd,v West Indies,Home,2011-11-22
3,Y,lost,122 runs,lost,2nd,v Australia,Away,2011-12-26
4,Y,lost,inns & 68 runs,won,1st,v Australia,Away,2012-01-03


In [5]:
df.describe()

,Ashwin,Result,Margin,Toss,Bat,Opposition,Location,Start Date
count,85,85,85,85,85,85,85,85
unique,2,3,61,2,2,8,2,85
top,Y,won,-,lost,1st,v Australia,Home,2019-11-22 00:00:00
freq,70,47,16,45,46,20,43,1
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-11-06 00:00:00
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-11-22 00:00:00


## Pre-process data (Whatever you feel might be required)

### date column can be dropped

In [6]:
df.drop('Start Date', axis=1, inplace=True)

In [7]:
df.head()

,Ashwin,Result,Margin,Toss,Bat,Opposition,Location
0,Y,won,5 wickets,lost,2nd,v West Indies,Home
1,Y,won,inns & 15 runs,won,1st,v West Indies,Home
2,Y,draw,-,lost,2nd,v West Indies,Home
3,Y,lost,122 runs,lost,2nd,v Australia,Away
4,Y,lost,inns & 68 runs,won,1st,v Australia,Away


## Data Description

In [310]:
df.describe()

,Ashwin,Result,Margin,Toss,Bat,Opposition,Location
count,85,85,85,85,85,85,85
unique,2,3,61,2,2,8,2
top,Y,won,-,lost,1st,v Australia,Home
freq,70,47,16,45,46,20,43


# Construction of Bayesian Network goes here 

## 1. Solution for part 1 

In [101]:
def calculate_prior_probability(array):
    df_array=pd.DataFrame(array)
    column = list(df_array.columns)[0]
    df_ash=df_array[column].value_counts()
    count = len(array)
    dict = {}
    val_to_print = "{"
    for i, v in df_ash.items():
        #val_to_print =val_to_print + "'" + i + "': "+ str(v) + "/" + str(count) + ", "
        dict[i] = round(v/count , 2)   
    #print(val_to_print[0:len(val_to_print)-2] ,"}")
    return dict

In [102]:
prob_dict = calculate_prior_probability(df.Ashwin)
print(prob_dict)

{'Y': 0.82, 'N': 0.18}


## 2. Solution for part 2 

In [156]:
#list of array is passed. last array will be used to compute the posteriro prob with previous arrays
def conditional_probability(list_of_arrys):
    length_of_list = len(list_of_arrys)    
    #print("lenght of list:",length_of_list)
    
    # Create the pandas DataFrame 
    #df = pd.DataFrame(list_of_arrys, columns = [column_name]) 
    #df = pd.DataFrame({"col0":list_of_arrys[0],"col1":list_of_arrys[1]})
    df = pd.DataFrame()
    i=0
    while i< length_of_list:
        df["col_"+str(i)] = list_of_arrys[i]
        i = i + 1
    
    #list of dataframe with posterior computation
    list_df=[]
    i=0
    last_index = length_of_list-1
    #print("last index:",last_index)
    
    #i_probs = df.groupby('col_2').size().div(len(df))
    last_col = "col_"+ str(last_index)
    #cond1 = pd.DataFrame(df.groupby([last_index, 0]).size().div(len(df)).div(i_probs, axis=0, level=0)).reset_index()
    #cond1 = pd.DataFrame(df.groupby(['col_2', 'col_0']).size().div(len(df)).div(i_probs, axis=0, level='col_0')).reset_index()
    while i < (length_of_list-1):
        col = "col_" + str(i)
        i_probs = df.groupby(col).size().div(len(df))
        cond = pd.DataFrame(df.groupby([last_col, col]).size().div(len(df))
                            .div(i_probs, axis=0, level=col)).reset_index()
        #print("iteration:", str(i))
        #print(cond)
        i = i+1
        list_df.append(cond)
        
    return list_df
    
    

In [157]:
def get_probility_combination(mat_1 , mat_2):
    prob_combination_list = []
    for i in mat_1:
        list1 = []
        #print(type(list1))
        if type(i)==list:
            list1 = i
        else:
            list1.append(i)
        for j in mat_2:
            list2 = list1.copy()
            list2.append(j)
            tmp = [i , j]
            prob_combination_list.append(list2)
            #prob_combination_list.append(list1.append(j))
    return prob_combination_list
    

In [158]:
def get_prob_combination_list(list_arr):
    #iterate the list of dataframe to create probbaility combination
    last_list_array_index = len(list_arr)-1
    prob_combination_list = get_probility_combination(list_arr[last_list_array_index].unique() ,
                                                      list_arr[last_list_array_index-1].unique())
    i = last_list_array_index-2
    #print(i)
    #print(prob_combination_list)
    while(i>=0):
        prob_combination_list = get_probility_combination(prob_combination_list , list_arr[i].unique())
        i = i-1
    return prob_combination_list


In [159]:
def get_prob_final_frame(prob_combination_list):
    index = 0
    df_final = pd.DataFrame()
    #sub_list has last array will put last array column 
    i = len(prob_combination_list[0])-1
    #print(i)
    for sub_list in prob_combination_list:
        j=0
        while(j<len(sub_list)):
            df_final.at[index,"col_"+str(i-j)] = sub_list[j]
            j = j+1   
        df_final.at[index,"prob"] = 0
        index +=1
    return df_final

In [160]:
def get_final_calculation_for_prob(df_retrun , list_arr , df_final):
    df_retrun = conditional_probability(list_arr)
    #print(df_retrun)
    i=0
    while(i<len(df_final)):
        for cond in df_retrun:
            col_names = list(cond.columns)
            column_0 = col_names[0]
            column_1 = col_names[1]
            column_2 = col_names[2]
            #print("names-",column_0,column_1,column_2)
            for index, row in cond.iterrows():
                #print(df_final.loc[i][column_0],row[column_0],df_final.loc[i][column_1],row[column_1])
                if df_final.loc[i][column_0]==row[column_0] and df_final.loc[i][column_1]==row[column_1]:
                    #df_final.at[i,"prob"] = df_final.loc[i]["prob"] * row[column_2]
                    #print(row[column_0] , row[column_1])
                    if df_final.loc[i]["prob"]==0:
                        df_final.at[i,"prob"] = row[column_2] #* prob_dict.get(row[column_0])
                        #print(df_final.loc[i]["prob"] , row[column_2] , prob_dict.get(row[column_0]))
                    else:
                        df_final.at[i,"prob"] = df_final.loc[i]["prob"] * row[column_2]#* prob_dict.get(row[column_0])
                        #print("else-",df_final.loc[i]["prob"] , df_final.loc[i]["prob"] , prob_dict.get(row[column_0]))
                    #df_final.at[i,"prob"] = df_final.loc[i]["prob"] * row[column_2]
                    #print(df_final.at[i,"prob"])
                    break

        i +=1
    #print(df_final)
    i=0
    col_names = list(df_final.columns)
    #print(col_names)
    col_for_prob = col_names[0]
    while i<len(df_final):
        df_final.at[i,"prob"] = df_final.loc[i]["prob"] * prob_dict.get(df_final.loc[i][col_for_prob])
        i = i+1

    #print(df_final)

    final_pomo = []
    col_names = list(df_final.columns)
    col_for_prob = col_names[0]
    prob_col = col_names[len(col_names)-1]
    col_names.pop(0)
    col_names.pop(len(col_names)-1)
    i=0
    while i<len(df_final):
        tmp=[]
        for eachCol in col_names:
            tmp.append(df_final.loc[i][eachCol])
        tmp.append(df_final.loc[i][col_for_prob])
        tmp.append(df_final.loc[i][prob_col])
        final_pomo.append(tmp)
        i = i + 1

    #print(final_pomo)
    
    return final_pomo

### list_arr needs to be modified to have more arrays

In [210]:
#create list of arrays to pass for conditional probability
#list_arr can be added wit more arrays to calculate
list_arr = []
list_arr.append(df.Result)#first array
list_arr.append(df.Bat)#secod array
#list_arr.append(df.Toss)#second array
#list_arr.append(df.Location)#first array
list_arr.append(df.Ashwin)#last array
#list_arr.append(df.Ashwin)#first array
#list_arr.append(df.Toss)#second array
#list_arr.append(df.Result)#third array
print(list_arr[0][0])
print(list_arr[1][0])
#print(list_arr[2][0])

won
2nd


In [211]:
prob_combination_list = get_prob_combination_list(list_arr)
df_final = get_prob_final_frame(prob_combination_list)
df_retrun = conditional_probability(list_arr)
final_map = get_final_calculation_for_prob(df_retrun , list_arr , df_final)

print(prob_combination_list)
print(df_final)
print(final_map)


[['Y', '2nd', 'won'], ['Y', '2nd', 'draw'], ['Y', '2nd', 'lost'], ['Y', '1st', 'won'], ['Y', '1st', 'draw'], ['Y', '1st', 'lost'], ['N', '2nd', 'won'], ['N', '2nd', 'draw'], ['N', '2nd', 'lost'], ['N', '1st', 'won'], ['N', '1st', 'draw'], ['N', '1st', 'lost']]
   col_2 col_1 col_0      prob
0      Y   2nd   won  0.620033
1      Y   2nd  draw  0.563750
2      Y   2nd  lost  0.473077
3      Y   1st   won  0.589399
4      Y   1st  draw  0.535897
5      Y   1st  lost  0.449704
6      N   2nd   won  0.002946
7      N   2nd  draw  0.005192
8      N   2nd  lost  0.008811
9      N   1st   won  0.003747
10     N   1st  draw  0.006603
11     N   1st  lost  0.011206
[['2nd', 'won', 'Y', 0.6200327332242226], ['2nd', 'draw', 'Y', 0.5637500000000002], ['2nd', 'lost', 'Y', 0.4730769230769231], ['1st', 'won', 'Y', 0.5893987049028676], ['1st', 'draw', 'Y', 0.5358967391304349], ['1st', 'lost', 'Y', 0.44970355731225287], ['2nd', 'won', 'N', 0.0029459901800327325], ['2nd', 'draw', 'N', 0.00519230769230769

## Solution 3
### Use the functions created above and python’s pomegranate library to define conditionality between variables to create Bayesian Network

In [212]:
def calculate_prior_probability_list(array):
    df_array=pd.DataFrame(array)
    column = list(df_array.columns)[0]
    df_ash=df_array[column].value_counts()
    count = len(array)
    dict = {}
    val_to_print = "{"
    for i, v in df_ash.items():
        val_to_print =val_to_print + "'" + i + "': "+ str(v) + "./" + str(count) + ", "
        dict[i] = v/count    
    val_to_print = val_to_print[0:len(val_to_print)-2] ,"}"
    return val_to_print

In [189]:
import matplotlib.pyplot as plt
import seaborn; seaborn.set_style('whitegrid')
import numpy

from pomegranate import *

numpy.random.seed(0)
numpy.set_printoptions(suppress=True)

%load_ext watermark
%watermark -m -n -p numpy,scipy,pomegranate

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Sun Mar 15 2020 

numpy 1.16.5
scipy 1.4.1
pomegranate 0.11.1

compiler   : MSC v.1915 64 bit (AMD64)
system     : Windows
release    : 10
machine    : AMD64
processor  : Intel64 Family 6 Model 142 Stepping 9, GenuineIntel
CPU cores  : 4
interpreter: 64bit


In [213]:
#prob_dict_loc = calculate_prior_probability(df.Location)
#prob_dict_ash = calculate_prior_probability(df.Ashwin)
#prob_dict_toss = calculate_prior_probability(df.Toss)
prob_dict_res = calculate_prior_probability(df.Result)
prob_dict_bat = calculate_prior_probability(df.Bat)
#print(prob_dict_bat,prob_dict_res)

#loc = DiscreteDistribution(prob_dict_loc)
#toss = DiscreteDistribution(prob_dict_toss)
bat = DiscreteDistribution(prob_dict_bat)
res = DiscreteDistribution(prob_dict_res)

#['draw', '2nd', 'won', 'Away', 'N'
# Ashwin is dependent on locaion , toss , bat ,result. 
#ashwin_playing = ConditionalProbabilityTable(final_map, [loc , toss , bat, res ])
ashwin_playing = ConditionalProbabilityTable(final_map, [ bat, res ])

In [221]:
# State objects hold both the distribution, and a high level name.
s1 = State(bat, name="bat")
s2 = State(res, name="res")
#s3 = State(toss, name="toss")
#s4 = State(loc, name="loc")
s5 = State(ashwin_playing, name="ashwin_playing")

In [222]:
# Create the Bayesian network object with a useful name
model = BayesianNetwork("Ashwin and match")

# Add the three states to the network 
#model.add_states(s1, s2, s3 , s4 , s5)
model.add_states(s1, s2 , s5)

In [223]:
# Add edges which represent conditional dependencies, where the second node is 
# conditionally dependent on the first node (Monty is dependent on both guest and prize)
model.add_edge(s1, s5)
model.add_edge(s2, s5)
#model.add_edge(s3, s5)
#model.add_edge(s4, s5)

#batting depends on toss
#model.add_edge(s2, s3)

In [224]:
model.bake()

## Solution for part 4

In [225]:
#Predicting Probabilities
# a) India winning, batting 2nd, Ashwin playing
#model.predict_proba([None ,None , '2nd', 'won' ,'Y'])
model.probability(['2nd', 'won' ,'Y'])

0.15686828150572832

In [226]:
# b) India winning, batting 2nd, Ashwin not playing
model.probability(['2nd', 'won' ,'N'])

0.0007453355155482819

In [227]:
# c) India losing, batting 2nd, Ashwin playing
model.probability(['2nd', 'lost' ,'Y'])

0.05658000000000002

In [228]:
# d) India losing, batting 2nd, Ashwin not playing
model.probability(['2nd', 'lost' ,'N'])

0.0010538181818181814

In [ ]:
#Feel free to add cells where necessary. 

<h3><center> Happy Coding!</center></h3>